In [1]:
import pandas as pd
import numpy as np
import os
import glob
from collections import Counter
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil import parser
import concurrent.futures
import time
import csv
import re
import os
import sys
import csv
headers = ['gkg_id', 'date', 'source', 'source_name', 'doc_id', 
        'themes', 'locations', 'persons', 'orgs', 
        'tone', 'pos', 'neg', 'polarity', 'ard', 'srd',
        'wc', 
        'lexicode_neg', 'lexicode_pos'#, # c3.*
#         'MACROECONOMICS', 'ENERGY', 'FISHERIES', 
#         'TRANSPORTATION', 'CRIME', 'SOCIAL_WELFARE',
#         'HOUSING', 'FINANCE', 'DEFENCE', 'SSTC',
#         'FOREIGN_TRADE', 'CIVIL_RIGHTS', 
#         'INTL_AFFAIRS', 'GOVERNMENT_OPS',
#         'LAND-WATER-MANAGEMENT', 'CULTURE',
#         'PROV_LOCAL', 'INTERGOVERNMENTAL',
#         'CONSTITUTIONAL_NATL_UNITY', 'ABORIGINAL',
#         'RELIGION', 'HEALTHCARE', 'AGRICULTURE',
#         'FORESTRY', 'LABOUR', 'IMMIGRATION',
#         'EDUCATION', 'ENVIRONMENT',
#         'finstab_pos', 'finstab_neg', 'finstab_neutral',
#         'finsent_neg', 'finsent_pos', 'finsent_unc',
#         'opin_neg', 'opin_pos',
#         'sent_pos', 'sent_neg', 'sent_pol'
]

csv.field_size_limit(sys.maxsize)


131072

In [2]:
# glob.glob(f"../Gdelt-data/gdelt-nz/2015*")

In [3]:
def merge_csv(f, y):
    start = time.time()
    
    

    
    header_written = False
    ''' Pandas method takes a longer timer for merge and hence have made it in pure python to boost up'''
    start = time.time()
    with open(f'../gdelt_csv/gdelt_f_2015/gdelt_{f}_{y}.csv', 'w', newline="") as fout:                         # merge csv
        wout = csv.writer(fout, delimiter=',')
        files = [x for x in glob.glob(f"../Gdelt-data/gdelt-{f}/{y}*.csv") if x != f'gdelt_{f}_{y}.csv']
        for file in files:
            print("processing {}".format(file))
            with open(file) as fin:
                cr = csv.reader(fin,delimiter=',')
                if not header_written:
                    wout.writerow(headers)
                    header_written = True
                wout.writerows(cr) 
    
    
    end = time.time()
    csvv = len(glob.glob(f"../Gdelt-data/gdelt-{f}/{y}*.csv"))
    print(f"Finished processing {csvv} csv's for {f} which took {round(end-start,2)} seconds")

In [5]:
%%time
Year = [i for i in range(2019, 2022)]
# merge_csv('nz', 20)
# merge_csv('au', 202)
merge_csv('ca', 2020)
# for y in Year:
#     merge_csv('uk', y)

processing ../Gdelt-data/gdelt-ca/20200201113000.gkg.csv
processing ../Gdelt-data/gdelt-ca/20200525221500.gkg.csv
processing ../Gdelt-data/gdelt-ca/20200229100000.gkg.csv
processing ../Gdelt-data/gdelt-ca/20201122043000.gkg.csv
processing ../Gdelt-data/gdelt-ca/20200101203000.gkg.csv
processing ../Gdelt-data/gdelt-ca/20200303114500.gkg.csv
processing ../Gdelt-data/gdelt-ca/20200531173000.gkg.csv
processing ../Gdelt-data/gdelt-ca/20200229210000.gkg.csv
processing ../Gdelt-data/gdelt-ca/20200614061500.gkg.csv
processing ../Gdelt-data/gdelt-ca/20201123034500.gkg.csv
processing ../Gdelt-data/gdelt-ca/20200501214500.gkg.csv
processing ../Gdelt-data/gdelt-ca/20201227054500.gkg.csv
processing ../Gdelt-data/gdelt-ca/20201220043000.gkg.csv
processing ../Gdelt-data/gdelt-ca/20201014123000.gkg.csv
processing ../Gdelt-data/gdelt-ca/20200825161500.gkg.csv
processing ../Gdelt-data/gdelt-ca/20200616093000.gkg.csv
processing ../Gdelt-data/gdelt-ca/20200310081500.gkg.csv
processing ../Gdelt-data/gdelt-

In [4]:
col = ['date', 'source', 'source_name', 
        'themes', 'locations', 'orgs', 
        'tone', 'pos', 'neg', 'polarity', 'wc',
]

def read_files(f):
    start = time.time()
    custom_date_parser = lambda x: datetime.strptime(x, "%Y%m%d%H%M%S")
    chunk = pd.read_csv(f"../gdelt_csv/gdelt_{f}.csv", parse_dates=['date'], date_parser = custom_date_parser, usecols=col, chunksize=1000000)    

    df = pd.concat(chunk)
    del chunk
    df['month'] = df['date'].dt.strftime('%b')
    cats = ['Jan', 'Feb', 'Mar', 'Apr','May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    df['month'] = pd.Categorical(df['month'], ordered=True, categories=cats)
#     df['month'] = df['month'].astype('int8')
    num = df.select_dtypes('number').columns
    df[num] = df[num].apply(lambda x: pd.to_numeric(x, downcast='float'))
    
    df.loc[:, df.dtypes == 'object'] =\
    df.select_dtypes(['object'])\
    .apply(lambda x: x.astype('category'))

    end = time.time()
    print(f"Read csv with chunks for {f}: ",round(end-start,2),"sec")

    
    return df

In [5]:
%%time
# df_nz = read_files('nz')
# df_au = read_files('au')
# df_uk = read_files('uk')
# df_us = read_files('us')
# df_in = read_files('in')


Read csv with chunks for in:  28.77 sec
CPU times: user 27.8 s, sys: 957 ms, total: 28.8 s
Wall time: 28.8 s


In [5]:
print(df_nz.info())
print(df_au.info())
print(df_uk.info())
print(df_us.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 901499 entries, 0 to 901498
Data columns (total 19 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   gkg_id        901499 non-null  object        
 1   date          901499 non-null  datetime64[ns]
 2   source        901499 non-null  int64         
 3   source_name   901491 non-null  object        
 4   doc_id        901499 non-null  object        
 5   themes        775886 non-null  object        
 6   locations     901499 non-null  object        
 7   persons       786844 non-null  object        
 8   orgs          767003 non-null  object        
 9   tone          901498 non-null  float64       
 10  pos           901498 non-null  float64       
 11  neg           901498 non-null  float64       
 12  polarity      901498 non-null  float64       
 13  ard           901498 non-null  float64       
 14  srd           901498 non-null  float64       
 15  wc            901

In [6]:
print(df_in.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 872934 entries, 0 to 872933
Data columns (total 12 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         872934 non-null  datetime64[ns]
 1   source       872934 non-null  float32       
 2   source_name  872854 non-null  category      
 3   themes       756473 non-null  category      
 4   locations    872934 non-null  category      
 5   orgs         764399 non-null  category      
 6   tone         872934 non-null  float32       
 7   pos          872934 non-null  float32       
 8   neg          872934 non-null  float32       
 9   polarity     872934 non-null  float32       
 10  wc           872934 non-null  float32       
 11  month        872934 non-null  category      
dtypes: category(5), datetime64[ns](1), float32(6)
memory usage: 80.7 MB
None


In [1]:
# due to less RAM unable to save as 

In [ ]:
df_nz.to_feather('../gdelt_feather/df_nz.ft')


In [ ]:
df_au.to_feather('../gdelt_feather/df_au.ft')


In [ ]:
df_uk.to_feather('../gdelt_feather/df_uk.ft')


In [ ]:
df_us.to_feather('../gdelt_feather/df_us.ft')

In [2]:
df = pd.read_csv("../gdelt_csv/gdelt_nz.csv")

In [7]:
df.head()

,gkg_id,date,source,source_name,doc_id,themes,locations,persons,orgs,tone,...,finstab_neg,finstab_neutral,finsent_neg,finsent_pos,finsent_unc,opin_neg,opin_pos,sent_pos,sent_neg,sent_pol
0,20210201170000-180,20210201170000,1,adn.com,https://www.adn.com/nation-world/2021/02/01/ma...,TAX_DISEASE;TAX_DISEASE_CORONAVIRUS;GENERAL_HE...,"2#New York, United States#US#USNY#42.1497#-74....",ned lamont;robert wachter;joe biden;anthony s ...,centers for disease;georgetown university cent...,-1.545254,...,NaN,NaN,50.0,25.0,34.0,99.0,80.0,0.259338,0.307007,NaN
1,20210201170000-199,20210201170000,1,fandompost.com,https://www.fandompost.com/2021/02/01/35th-dig...,SOVEREIGNTY;TAX_FNCACT;TAX_FNCACT_DIRECTOR;TAX...,1#South Africa#SF#SF#-30#26#SF;1#Australia#AS#...,taichi yagami;atsuhiro tomioka;masato mitsuka;...,toei animation;twitter,-0.358423,...,NaN,NaN,5.0,1.0,NaN,7.0,7.0,0.252633,0.245836,NaN
2,20210201170000-228,20210201170000,1,abc11.com,https://abc11.com/timely-groundhog-day-funny-a...,TAX_FNCACT;TAX_FNCACT_GARDENER;WB_678_DIGITAL_...,"2#Delaware, United States#US#USDE#39.3498#-75....",jeff permar;punxsutawney phil,instagram;youtube,0.930233,...,NaN,NaN,4.0,5.0,2.0,12.0,23.0,0.252245,0.265234,NaN
3,20210201170000-373,20210201170000,1,stuff.co.nz,https://www.stuff.co.nz/entertainment/arts/124...,MANMADE_DISASTER_IMPLIED;TAX_WORLDLANGUAGES;TA...,"5#Taranaki, Taranaki, New Zealand#NZ#NZF9#-39....",tangata whenua;tiriti o waitangi;courtney john...,ministry for culture;museum in porirua;museum ...,-0.330033,...,NaN,NaN,20.0,7.0,7.0,26.0,36.0,0.275194,0.269655,NaN
4,20210201170000-386,20210201170000,1,theguardian.com,https://www.theguardian.com/environment/2021/f...,TAX_WORLDMAMMALS;TAX_WORLDMAMMALS_HUMAN;UNGP_F...,1#Mexico#MX#MX#23#-102#MX;1#Australia#AS#AS#-2...,corey bradshaw;tim doherty,deakin university;global ecology lab at flinde...,-2.151899,...,NaN,NaN,19.0,2.0,10.0,28.0,23.0,0.277778,0.266041,NaN
